# Preprocess The Data

In [14]:
# import libraraies
import pandas as pd

# load data
data = pd.read_csv('data.csv')

# remove unwanted columns
data = data.drop(columns=['ReasonsForLack'], axis=1)

# saving the unique values for lable encoding
values = {column: list(data[column].unique()) 
          for column in data.columns.values}

# Add a filed called is_stem which tells whether a student has taken a science degree or not
stem_fields = ['Engineering Sciences',
               'Medicine, Health Sciences',
               'Mathematics or Natural Sciences'] 

data['is_stem'] = data['EducationSector'].apply(lambda val: 1 if val in stem_fields else 0)
    
# Lable encode the following columns: IndividualProject, Gender, City, Influenced, MentalDisorder
columns_to_lable = ['IndividualProject', 'Gender', 'City', 'Influenced', 'MentalDisorder']

for column in columns_to_lable:
    data[column] = data[column].apply(lambda val: values[column].index(val)) 

# Onehot encode the following columns : EducationSector, KeyTraits 
dummies_ed_sec = pd.get_dummies(data['EducationSector'], prefix='degree')
dummies_traits = pd.get_dummies(data['KeyTraits'], prefix='trait')

data = data.join(dummies_ed_sec.join(dummies_traits)).drop(columns=['EducationSector', 'KeyTraits'])   

data.head()

,IndividualProject,Age,Gender,City,Influenced,Perseverance,DesireToTakeInitiative,Competitiveness,SelfReliance,StrongNeedToAchieve,...,degree_Language and Cultural Studies,degree_Mathematics or Natural Sciences,"degree_Medicine, Health Sciences",degree_Others,"degree_Teaching Degree (e.g., B.Ed)",trait_Passion,trait_Positivity,trait_Resilience,trait_Vision,trait_Work Ethic
0,0,19,0,0,0,2,2,3,3,2,...,0,0,0,0,0,1,0,0,0,0
1,1,22,0,1,1,3,3,3,4,4,...,0,0,0,0,0,0,0,0,1,0
2,0,18,0,0,0,3,4,3,3,3,...,0,0,0,0,0,1,0,0,0,0
3,1,20,0,0,1,3,3,3,3,4,...,0,0,0,0,0,0,0,1,0,0
4,1,19,0,0,1,2,3,3,3,4,...,0,0,0,0,0,0,0,0,1,0


# Split Data into training and testing data

In [392]:
from sklearn.model_selection import train_test_split

X_cols = data.columns.to_list()
X_cols.remove('y')
Y_col = 'y'

X,y = data[X_cols],data[Y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = .1)

# Training Models

We would be traning several models, and comparing 
them to select the one which has the good score on 
the data, both training and testing. 

The models we would be using are:

- Decision Trees
- Random Forest Classifier
- SVM
- Multi Perceptron (Dense) Neural Network

## Decision Tree

In [440]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [441]:
accuracy = clf.score(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 63.64


## Random Forest Classifier

In [448]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [449]:
accuracy = clf.score(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 59.09


## SVM

In [453]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification


clf = make_pipeline(StandardScaler(),
                    LinearSVC(tol=1e-5))
clf.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(tol=1e-05))])

In [454]:
accuracy = clf.score(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 63.64


## Multi Perceptron (Dense) Neural Networks

In [369]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(300, input_dim=len(X_cols), activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [370]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2930 - accuracy: 0.4967
Epoch 2/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2460 - accuracy: 0.5650
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2503 - accuracy: 0.4806
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 0.2447 - accuracy: 0.5661
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 0.2344 - accuracy: 0.5850
Epoch 6/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2377 - accuracy: 0.6132
Epoch 7/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2429 - accuracy: 0.5871
Epoch 8/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2253 - accuracy: 0.6135
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 0.2258 - accuracy: 0.6622
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 0.2271 - accuracy: 0.5918
Epoch 11/200
7/7 [===========

In [371]:
_, accuracy = model.evaluate(X_test, y_test)

print('Accuracy: %.2f' % (accuracy*100))


1/1 [==============================] - 0s 78ms/step - loss: 0.2765 - accuracy: 0.6818
Accuracy: 68.18


# Results 

- Decision Trees : 63.64%
- Random Forest Classifier : 59.09%
- SVM : 63.64%
- Multi Perceptron (Dense) Neural Network : 68.18%